In [1]:
from bac.simulate.namd import (Simulation, NonBondedController, PME, BondConstraint, AtomConstraint, 
                               HarmonicConstraint, non_bonded_controller, VelocityRescaling, VelocityReassignment,
                               LangevinDynamics, BerendsenPressureCoupling, FreeEnergyController)
from pathlib import Path

## Step 1: Minimize

In [2]:
minimize = Simulation(minimization=True)
minimize.timestep = 2
minimize.number_of_steps = 2500

minimize.temperature = 298.15
minimize.output_energies = 100
minimize.output_pressure = 100
minimize.name = 'min'
minimize.output_name = 'min'
minimize.binary_output = False

minimize.amber = True
minimize.coordinates = 'mobley_9055303.pdb'
minimize.parameters = 'mobley_9055303.prmtop'
minimize.read_exclusions = False

In [3]:
nbc = NonBondedController()

nbc.excluded_interactions = non_bonded_controller.Interaction.scaled_one_four
nbc.one_four_scaling = 0.833333

nbc.cutoff = 12
nbc.switching = True
nbc.switching_distance = 9
nbc.pairlist_distance = 12

nbc.pme = PME(grid_spacing=1.2)
nbc.pme.interpolation_order = 6

nbc.wrap_water, nbc.wrap_all = True, True
nbc.cell_basis_vector_1 = 31.7894, 0.0, 0.0
nbc.cell_basis_vector_2 = 0.0, 31.7894, 0.0
nbc.cell_basis_vector_3 = 0.0, 0.0, 31.7894

minimize.non_bonded_controller = nbc

In [4]:
minimize.temperature_controller = LangevinDynamics(temperature=298.15, damping=5, applies_to_hydrogen=False)

In [5]:
minimize.constraints.bond_constraint = BondConstraint(bonds='all', tolerance=0.00001, iterations=100)

In [6]:
fe = FreeEnergyController()
fe.type = 'ti'
fe.file = 'mobley_9055303.pdb'
fe.output_frequency = 100

fe.van_der_waals_shift_coefficient = 5
fe.electronic_interaction_start_lambda = 0.45
fe.van_der_waals_end_lambda = 1.0
fe.decouple = True

fe.start_lambda = 0.0
fe.end_lambda = 0.0

minimize.free_energy_controller = fe


## NVT equilibration

In [7]:
eq1 = next(minimize)

In [8]:
eq1.number_of_steps = 25000
eq1.minimization = False
eq1.name = 'eq1'
eq1.output_name = 'eq1'


eq1.restart_frequency = 12500
eq1.non_bonded_controller.xst_frequency = 2500
eq1.dcd_frequency = 5000

# NPT equilibration

In [9]:
eq2 = next(eq1)
eq2.name = 'eq2'
eq2.output_name = 'eq2'

In [10]:
eq2.pressure_controller = BerendsenPressureCoupling(target=1, compressibility=4.57e-5, relaxation_time=100, 
                                                              frequency=2, use_group_pressure=True)
eq2.restart_frequency = 5000

/Users/kristofarkas/Developer/BAC2/bac/utils/decorators.py:99: UserWarning: 'BerendsenPressureCoupling' object has no attribute 'simulation'
  warnings.warn(f'{e}')


# Production

In [11]:
prod = next(eq2)

prod.name = 'prod'
prod.output_name = 'prod'
prod.number_of_steps = 250_000
prod.restart_frequency = 25000

# Workflow

In [12]:
from bac.simulate import Workflow, Replica, LambdaWindow

In [13]:
wf = Workflow(resource='bw', r_dir='mobley_9055303')

In [14]:
wf.simulations = [minimize, eq1, eq2, prod]

In [15]:
wf.ensembles = [Replica(number_of=5), LambdaWindow(number_of_windows=11)]

In [16]:
wf.preprocess_simulations()

In [21]:
print("module load rca\n" \
                    "module load craype-hugepages8M\n" \
                    "module list\n" \
                    "export HUGETLB_DEFAULT_PAGE_SIZE=8M\n" \
                    "export HUGETLB_MORECORE=no\n" \
                    "export ATP_ENABLED=1\n" \
                    "ulimit -c unlimited\n")

module load rca
module load craype-hugepages8M
module list
export HUGETLB_DEFAULT_PAGE_SIZE=8M
export HUGETLB_MORECORE=no
export ATP_ENABLED=1
ulimit -c unlimited

